In [1]:
from pathlib import Path
# Create a simplified GBNF grammar for testing
# Focus on: NOP, END, MOVJ, MOVL, SET (no complex features)

simplified_gbnf = r'''
root ::= program_header statement* program_footer

program_header ::= "NOP" ws

program_footer ::= "END"

statement ::= ws (motion_statement | set_statement | label_definition | comment) ws

motion_statement ::= movj_statement | movl_statement

movj_statement ::= "MOVJ" ws position ws "VJ=" number ws "ACC=" number ws "CNT=" number

movl_statement ::= "MOVL" ws position ws "VL=" number ws "ACC=" number ws "CNT=" number

set_statement ::= "SET" ws variable ws number

label_definition ::= [A-Z] [0-9]+

position ::= "P" [0-9]+

variable ::= ("I" | "R") [0-9]+

number ::= [0-9]+ ("." [0-9]+)?

comment ::= "//" [^\n]*

ws ::= [ \t\n\r]*
'''.strip()

print("Simplified GBNF Grammar:")
print("=" * 60)
print(simplified_gbnf)
print("=" * 60)
print(f"\nRules: {simplified_gbnf.count('::=')}")
print("\nThis simplified grammar includes:")
print("  ✓ NOP/END structure")
print("  ✓ MOVJ with required parameters (VJ, ACC, CNT)")
print("  ✓ MOVL with required parameters (VL, ACC, CNT)")
print("  ✓ SET for variables (I1, R1, etc.)")
print("  ✓ Labels (L10, L20, etc.)")
print("  ✓ Comments")

# Save to file
simplified_file = Path('tree-sitter-siasun/siasun_robot_simple.gbnf')
simplified_file.parent.mkdir(exist_ok=True)
with open(simplified_file, 'w') as f:
    f.write(simplified_gbnf)
print(f"\n✓ Saved to: {simplified_file}")

Simplified GBNF Grammar:
root ::= program_header statement* program_footer

program_header ::= "NOP" ws

program_footer ::= "END"

statement ::= ws (motion_statement | set_statement | label_definition | comment) ws

motion_statement ::= movj_statement | movl_statement

movj_statement ::= "MOVJ" ws position ws "VJ=" number ws "ACC=" number ws "CNT=" number

movl_statement ::= "MOVL" ws position ws "VL=" number ws "ACC=" number ws "CNT=" number

set_statement ::= "SET" ws variable ws number

label_definition ::= [A-Z] [0-9]+

position ::= "P" [0-9]+

variable ::= ("I" | "R") [0-9]+

number ::= [0-9]+ ("." [0-9]+)?

comment ::= "//" [^\n]*

ws ::= [ \t\n\r]*

Rules: 14

This simplified grammar includes:
  ✓ NOP/END structure
  ✓ MOVJ with required parameters (VJ, ACC, CNT)
  ✓ MOVL with required parameters (VL, ACC, CNT)
  ✓ SET for variables (I1, R1, etc.)
  ✓ Labels (L10, L20, etc.)
  ✓ Comments

✓ Saved to: tree-sitter-siasun/siasun_robot_simple.gbnf


In [3]:
# Test with simplified grammar
import ollama

try:
    print("Testing with SIMPLIFIED grammar...")
    print("=" * 60)
    
    response = ollama.generate(
        model='qwen3-coder',
        prompt='NOP\nMOVJ P1 VJ=50 ACC=100 CNT=0\nEND',  # Provide a valid example as seed
        options={
            'grammar': simplified_gbnf,
            'temperature': 0.1,  # Very low for strict adherence
            'num_predict': 100,
            'seed': 42  # Reproducible results
        }
    )
    
    generated = response['response']
    print("Generated:")
    print(generated)
    print("=" * 60)
    
    # Parse and validate
    from tree_sitter import Language, Parser
    import ctypes
    
    # Load parser
    lib = ctypes.CDLL('build/siasun_robot.dylib')
    tree_sitter_siasun_robot = lib.tree_sitter_siasun_robot
    tree_sitter_siasun_robot.restype = ctypes.c_void_p
    SIASUN_LANG = Language(tree_sitter_siasun_robot())
    parser = Parser(SIASUN_LANG)
    
    # Parse generated code
    tree = parser.parse(generated.encode('utf-8'))
    
    def count_errors(node):
        """Count ERROR nodes recursively"""
        count = 1 if node.type == 'ERROR' else 0
        for child in node.children:
            count += count_errors(child)
        return count
    
    error_count = count_errors(tree.root_node)
    
    if error_count == 0:
        print("✓ SUCCESS: Generated code is valid!")
    else:
        print(f"⚠️  {error_count} parse error(s) found")
        print("\nGenerated code has issues. The model is not respecting the grammar.")
        
except Exception as e:
    print(f"Error: {e}")

Testing with SIMPLIFIED grammar...
Generated:
This appears to be a robot programming code snippet, likely from a Fanuc or similar industrial robot controller. Let me break down each command:

## Code Analysis:

**NOP** - No Operation
- This is a placeholder instruction that does nothing
- Often used for debugging or as a breakpoint

**MOVJ P1 VJ=50 ACC=100 CNT=0**
- **MOVJ** - Move Joints (joint interpolation)
- **P1** - Target
⚠️  15 parse error(s) found

Generated code has issues. The model is not respecting the grammar.


/var/folders/sy/zg00xvpj2597p3wtxcmy_c100000gp/T/ipykernel_16876/2730943104.py:32: DeprecationWarning: int argument support is deprecated
  SIASUN_LANG = Language(tree_sitter_siasun_robot())


## Solution: Create Simplified GBNF Grammar

The full grammar (52 rules) is too complex for reliable constraint enforcement. Let's create a minimal subset with just core commands:

In [4]:
# Check GBNF grammar file validity
from pathlib import Path

grammar_file = Path('tree-sitter-siasun/siasun_robot.gbnf')

if grammar_file.exists():
    with open(grammar_file, 'r') as f:
        grammar_content = f.read()
    
    print(f"✓ Grammar file found: {grammar_file}")
    print(f"  Size: {len(grammar_content)} characters")
    print(f"  Rules: {grammar_content.count('::=')} total")
    print(f"\nFirst 500 characters:")
    print("=" * 60)
    print(grammar_content[:500])
    print("=" * 60)
    
    # Check for common GBNF issues
    issues = []
    if '::= ::=' in grammar_content:
        issues.append("⚠️  Double rule separator detected")
    if grammar_content.count('(') != grammar_content.count(')'):
        issues.append("⚠️  Unbalanced parentheses")
    if grammar_content.count('[') != grammar_content.count(']'):
        issues.append("⚠️  Unbalanced brackets")
    
    if issues:
        print("\n🔴 Potential issues:")
        for issue in issues:
            print(f"  {issue}")
    else:
        print("\n✓ No obvious syntax issues detected")
else:
    print(f"❌ Grammar file not found: {grammar_file}")
    print("   Run grammar_to_gbnf.py first to generate the GBNF file")

✓ Grammar file found: tree-sitter-siasun/siasun_robot.gbnf
  Size: 3460 characters
  Rules: 52 total

First 500 characters:
root ::= (_statement)*

_statement ::= (program_header | program_footer | label_definition | motion_statement | logic_statement | io_statement | control_statement | macro_statement | socket_statement)
_value ::= (number | string_literal | variable | bool_value | identifier)
analog_io_statement ::= ("AIN" | "AOUT") (var_r "AI\\d+" | "AO\\d+" "=" _value)
assignment ::= "SET" variable ("=" | "") _value ("N=" _value (variable | "") | "")
bool_value ::= ("ON" | "OFF" | "TRUE" | "FALSE")
call_statemen

✓ No obvious syntax issues detected


# GBNF Grammar Debugging

Before using the grammar with Ollama, let's verify:
1. The GBNF file is correctly formatted
2. Ollama version supports grammar constraints
3. Create a simplified grammar subset for testing

In [5]:
from tree_sitter import Language, Parser


In [6]:
def walk_tree(node, source_bytes, level=0):
    """
    Recursively walks the Abstract Syntax Tree (AST) and prints nodes.
    """
    indent = "  " * level
    
    # Extract the actual text for this node
    node_text = source_bytes[node.start_byte:node.end_byte].decode('utf-8').strip()
    
    # Only print relevant nodes (skip pure whitespace if desired)
    if node_text:
        # Print format: [Node Type] : Content
        print(f"{indent}[{node.type}] : {node_text.splitlines()}...")

    # Recurse for children
    for child in node.children:
        walk_tree(child, source_bytes, level + 1)


In [7]:
# 1. Load the compiled language
# For tree-sitter >= 0.25.0, we need to use ctypes to load the library
import ctypes

# Load the shared library
lib = ctypes.CDLL('build/siasun_robot.dylib')

# Get the language function - must match the grammar name
tree_sitter_siasun_robot = lib.tree_sitter_siasun_robot
tree_sitter_siasun_robot.restype = ctypes.c_void_p

# Create Language object with the pointer from the library
SIASUN_LANG = Language(tree_sitter_siasun_robot())

/var/folders/sy/zg00xvpj2597p3wtxcmy_c100000gp/T/ipykernel_16876/3683889081.py:13: DeprecationWarning: int argument support is deprecated
  SIASUN_LANG = Language(tree_sitter_siasun_robot())


In [8]:
# 2. Initialize the parser
parser = Parser(SIASUN_LANG)


In [9]:

# 3. Define a sample Siasun Robot Program
# Based on the manual:
# - NOP/END structure [1]
# - MOVJ/MOVL motion commands [2, 3]
# - SET assignment [4]
# - IF logic and Labels [5, 6]
# - I/O Control [7]
source_code = b"""
NOP

// Initialize Variables
SET I1 0
SET R1 10.5

// Label Definition
L10

// Motion Instructions
MOVJ P1 VJ=50 ACC=100 CNT=0
MOVL P2 VL=600 ACC=50 CNT=1

// IO Operation
OUT OT#1=ON

// Logic Control
ADD I1 1
IF I1 < 5 L10

// Conditional Call
IF I1 = 5 CALL SUB1

END
"""


In [10]:

# 4. Parse the code
tree = parser.parse(source_code)
root_node = tree.root_node


In [11]:

# 5. Visualize the Syntax Tree
print(f"Root Type: {root_node.type}")
print("-" * 40)

# Recursive function to traverse and print the tree
walk_tree(root_node, source_code)

Root Type: source_file
----------------------------------------
[source_file] : ['NOP', '', '// Initialize Variables', 'SET I1 0', 'SET R1 10.5', '', '// Label Definition', 'L10', '', '// Motion Instructions', 'MOVJ P1 VJ=50 ACC=100 CNT=0', 'MOVL P2 VL=600 ACC=50 CNT=1', '', '// IO Operation', 'OUT OT#1=ON', '', '// Logic Control', 'ADD I1 1', 'IF I1 < 5 L10', '', '// Conditional Call', 'IF I1 = 5 CALL SUB1', '', 'END']...
  [program_header] : ['NOP']...
  [comment] : ['// Initialize Variables']...
  [logic_statement] : ['SET I1 0']...
    [assignment] : ['SET I1 0']...
      [SET] : ['SET']...
      [variable] : ['I1']...
        [var_i] : ['I1']...
      [number] : ['0']...
  [logic_statement] : ['SET R1 10.5']...
    [assignment] : ['SET R1 10.5']...
      [SET] : ['SET']...
      [variable] : ['R1']...
        [var_r] : ['R1']...
      [number] : ['10.5']...
  [comment] : ['// Label Definition']...
  [label_definition] : ['L10']...
  [comment] : ['// Motion Instructions']...
  [mot

# Using GBNF Grammar with Ollama

Ollama supports GBNF (GGML BNF) grammars for constrained generation. This notebook demonstrates three methods to use the Siasun robot GBNF grammar with Ollama.

**Prerequisites:**
- Ollama installed and running
- A code-focused model (e.g., `codellama`, `deepseek-coder`)
- The GBNF grammar file generated from the Siasun grammar

**Note:** GBNF support requires a recent version of Ollama. Check your version with `ollama version`.

## Method 1: Python API (ollama-python)

This method uses the `ollama` Python package to generate Siasun robot programs with grammar constraints.

In [12]:
# Install ollama-python if not already installed
# !pip install ollama

import ollama
from pathlib import Path

# Load the GBNF grammar file
grammar_file = Path('tree-sitter-siasun/siasun_robot.gbnf')

with open(grammar_file, 'r') as f:
    siasun_grammar = f.read()

print(f"Loaded GBNF grammar from: {grammar_file}")
print(f"Grammar size: {len(siasun_grammar)} characters")
print(f"Grammar rules: {siasun_grammar.count('::=')} rules")

Loaded GBNF grammar from: tree-sitter-siasun/siasun_robot.gbnf
Grammar size: 3460 characters
Grammar rules: 52 rules


In [13]:
# Generate a Siasun robot program using the grammar constraint
# This ensures the output follows the Siasun language syntax

try:
    response = ollama.generate(
        # model='codellama',  # Use a code-focused model
        model='qwen3-coder',  # Use a code-focused model
        # CRITICAL: Prompt must be directive - only request code, no explanations
        prompt='Write only valid Siasun robot code (NOP...END) for pick and place. No comments, no explanations, just code:',
        options={
            'grammar': siasun_grammar,
            'temperature': 0.3,  # Lower temperature for stricter grammar adherence
            'num_predict': 200,  # Limit output length
            'top_p': 0.9
        }
    )
    
    print("Generated Siasun Robot Program:")
    print("=" * 60)
    print(response['response'])
    print("=" * 60)
    
except Exception as e:
    print(f"Error: {e}")
    print("Make sure Ollama is running and the model is available.")
    print("Run: ollama pull codellama")

Generated Siasun Robot Program:
NOP
MOV PR[1],P[1]
MOV PR[2],P[2]
MOV PR[3],P[3]
MOV PR[4],P[4]
MOV PR[5],P[5]
MOV PR[6],P[6]
MOV PR[7],P[7]
MOV PR[8],P[8]
MOV PR[9],P[9]
MOV PR[10],P[10]
MOV PR[11],P[11]
MOV PR[12],P[12]
MOV PR[13],P[13]
MOV PR[14],P[14]
MOV PR[15],P[15]
MOV PR[16],P[16]
MOV PR[17],P[17]
MOV PR[18],P[18]
MOV PR[19],P[19]
MOV PR[20],


In [14]:
# Validate the generated program using our Siasun interpreter
# This verifies that the grammar-constrained output is syntactically correct

generated_program = response['response'] if 'response' in locals() else None

if generated_program:
    # Parse with tree-sitter
    generated_tree = parser.parse(generated_program.encode('utf-8'))
    
    print("Parse Tree:")
    print("-" * 60)
    walk_tree(generated_tree.root_node, generated_program.encode('utf-8'))
    
    # Check for syntax errors
    def has_errors(node):
        """Recursively check for ERROR nodes in the parse tree"""
        if node.type == 'ERROR':
            return True
        return any(has_errors(child) for child in node.children)
    
    if has_errors(generated_tree.root_node):
        print("\n⚠️  Parse errors detected in generated program")
    else:
        print("\n✓ Generated program is syntactically valid!")

Parse Tree:
------------------------------------------------------------
[ERROR] : ['NOP', 'MOV PR[1],P[1]', 'MOV PR[2],P[2]', 'MOV PR[3],P[3]', 'MOV PR[4],P[4]', 'MOV PR[5],P[5]', 'MOV PR[6],P[6]', 'MOV PR[7],P[7]', 'MOV PR[8],P[8]', 'MOV PR[9],P[9]', 'MOV PR[10],P[10]', 'MOV PR[11],P[11]', 'MOV PR[12],P[12]', 'MOV PR[13],P[13]', 'MOV PR[14],P[14]', 'MOV PR[15],P[15]', 'MOV PR[16],P[16]', 'MOV PR[17],P[17]', 'MOV PR[18],P[18]', 'MOV PR[19],P[19]', 'MOV PR[20],']...
  [program_header] : ['NOP']...
  [identifier] : ['MOV']...
  [identifier] : ['PR']...
  [ERROR] : ['[']...
  [number] : ['1']...
  [ERROR] : [']']...
  [,] : [',']...
  [identifier] : ['P']...
  [ERROR] : ['[']...
  [number] : ['1']...
  [ERROR] : [']']...
  [identifier] : ['MOV']...
  [identifier] : ['PR']...
  [ERROR] : ['[']...
  [number] : ['2']...
  [ERROR] : [']']...
  [,] : [',']...
  [identifier] : ['P']...
  [ERROR] : ['[']...
  [number] : ['2']...
  [ERROR] : [']']...
  [identifier] : ['MOV']...
  [identifier] : 

## Method 2: REST API (curl)

This method uses Ollama's HTTP API to generate programs. Useful for integrating with other applications or languages.

In [ ]:
# Using Python's requests library to call Ollama REST API
import requests
import json

# Prepare the API request
ollama_url = "http://localhost:11434/api/generate"

# Create the request payload
payload = {
    "model": "codellama",
    "prompt": "Generate a Siasun robot program for pick and place operation",
    "stream": False,  # Get complete response at once
    "options": {
        "grammar": siasun_grammar,
        "temperature": 0.7,
        "num_predict": 300
    }
}

print("Sending request to Ollama REST API...")
print(f"URL: {ollama_url}")
print(f"Model: {payload['model']}")
print(f"Prompt: {payload['prompt']}")
print("-" * 60)

try:
    response = requests.post(ollama_url, json=payload, timeout=60)
    response.raise_for_status()
    
    result = response.json()
    generated_code = result.get('response', '')
    
    print("Generated Program:")
    print("=" * 60)
    print(generated_code)
    print("=" * 60)
    
    # Show generation statistics
    print(f"\nGeneration stats:")
    print(f"  Total duration: {result.get('total_duration', 0) / 1e9:.2f}s")
    print(f"  Load duration: {result.get('load_duration', 0) / 1e9:.2f}s")
    print(f"  Eval count: {result.get('eval_count', 0)} tokens")
    
except requests.exceptions.ConnectionError:
    print("Error: Cannot connect to Ollama. Is it running?")
    print("Start Ollama with: ollama serve")
except requests.exceptions.Timeout:
    print("Error: Request timed out. Try a simpler prompt or increase timeout.")
except Exception as e:
    print(f"Error: {e}")

### Equivalent curl command

The same request can be made from the terminal using curl:

In [ ]:
# Generate the curl command for terminal use
# Note: This is for documentation - run it in a terminal, not here

curl_command = f'''
curl http://localhost:11434/api/generate \\
  -H "Content-Type: application/json" \\
  -d '{{
    "model": "codellama",
    "prompt": "Generate a Siasun robot program",
    "stream": false,
    "options": {{
      "grammar": "<GRAMMAR_CONTENT>",
      "temperature": 0.7
    }}
  }}'
'''

print("Equivalent curl command:")
print(curl_command)
print("\nNote: Replace <GRAMMAR_CONTENT> with the actual grammar string")
print("Or load from file: --data-binary @tree-sitter-siasun/siasun_robot.gbnf")

## Method 3: CLI Method (if supported)

Some versions of Ollama may support grammar files directly via CLI. This is the simplest method when available.

In [ ]:
# Check if Ollama CLI supports grammar files
import subprocess
import shutil

# Check if ollama is installed
ollama_path = shutil.which('ollama')

if ollama_path:
    print(f"Ollama found at: {ollama_path}")
    
    # Check Ollama version
    try:
        version_result = subprocess.run(
            ['ollama', 'version'],
            capture_output=True,
            text=True,
            timeout=5
        )
        print(f"Version: {version_result.stdout.strip()}")
    except Exception as e:
        print(f"Could not get version: {e}")
    
    # Show example CLI usage (for documentation)
    print("\n" + "=" * 60)
    print("Example CLI usage (run in terminal):")
    print("=" * 60)
    
    cli_examples = [
        "# Method 1: Inline prompt",
        'ollama run codellama --grammar-file tree-sitter-siasun/siasun_robot.gbnf \\',
        '  "Generate a Siasun robot pick and place program"',
        "",
        "# Method 2: Interactive mode",
        "ollama run codellama --grammar-file tree-sitter-siasun/siasun_robot.gbnf",
        "",
        "# Note: --grammar-file flag may not be available in all versions",
        "# Use the Python API method instead if CLI method doesn't work"
    ]
    
    for line in cli_examples:
        print(line)
        
else:
    print("Ollama not found. Install it from: https://ollama.com")

## Comparison of Methods

| Method | Pros | Cons | Best For |
|--------|------|------|----------|
| **Python API** | Easy integration, good error handling | Requires Python package | Notebooks, Python apps |
| **REST API** | Language-agnostic, flexible | More verbose | Web services, integration |
| **CLI** | Simplest for testing | Limited in scripts | Quick testing, demos |

## Tips for Better Results

1. **Choose the right model**: Code-focused models work best
   - `codellama` - Good for code generation
   - `deepseek-coder` - Optimized for programming
   - `starcoder2` - Alternative code model

2. **Simplify complex grammars**: The full Siasun grammar is large (52 rules). For faster results:
   - Create a simplified subset of common commands
   - Focus on core features (NOP/END, MOVJ/MOVL, SET, IF)

3. **Adjust parameters**:
   - `temperature`: 0.5-0.8 for code (lower = more deterministic)
   - `num_predict`: Limit tokens to avoid long generation times
   - `top_p`: 0.9-0.95 for better diversity

4. **Provide context**: Include examples in the prompt for better results

In [ ]:
# Example: Generate with context-rich prompt
context_prompt = """Generate a Siasun robot program with the following structure:
1. Start with NOP
2. Set tool frame to #1
3. Define position P1 as current position
4. Move linearly by 100mm along X axis
5. End with END

Use proper Siasun syntax with motion parameters (VL, ACC, CNT) and OFFSET for relative movement."""

print("Enhanced Prompt:")
print(context_prompt)
print("\n" + "=" * 60)

# This prompt provides clear requirements, making it easier for the model
# to generate correct code even with grammar constraints